In [ ]:
#Importing and Modifying Training & Testing Data

#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

#Modules
import gensim.models
import pandas as pd
import torch
import numpy as np
from gensim.models import FastText, Word2Vec, KeyedVectors

#Deepti Import Data
dataset = pd.read_csv('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/medical_tc_train.csv')
testdataset = pd.read_csv('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/medical_tc_test.csv')

drive.mount("/content/drive", force_remount=True)
modelemb = Word2Vec.load('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/w2v_OA_CR_300d.bin')

dataset = pd.DataFrame.to_numpy(dataset)
testdataset = pd.DataFrame.to_numpy(testdataset)

#In training dataset, make each class have 1100 abstract samples to even out classes
dataset = dataset[dataset[:, 0].argsort()]
datasetnew=np.copy(dataset[:5500,:])

Dofstarts=dict()
for i in range(len(dataset)):
  row=dataset[i]
  if int(row[0]) not in Dofstarts:
    Dofstarts[int(row[0])] = i

for i in range(5):
  i = i+1
  valtostart=Dofstarts[i]
  datasetnew[(i-1)*1100:i*1100,:] = dataset[valtostart:valtostart+1100,:]

dataset = datasetnew

In [ ]:
pip install torch
pip install gensim
pip install torch.utils

In [ ]:
import torch.utils.checkpoint
import gensim.models

In [ ]:
#Importing and Modifying Training & Testing Data

from google.colab import drive
drive.mount('/content/drive')

#modules
import pandas as pd
import torch
import numpy as np

from gensim.models import FastText, Word2Vec, KeyedVectors

#Deepti Import Data
dataset = pd.read_csv('/content/drive/MyDrive/Deep Learning Project 2/medical_tc_train.csv')
testdataset = pd.read_csv('/content/drive/MyDrive/Deep Learning Project 2/medical_tc_test.csv')

#wordembeddings = pd.read_csv('/content/drive/MyDrive/Deep Learning Project 2/w2v_100d_oa_all.tar.gz')

drive.mount("/content/drive", force_remount=True)

modelemb = Word2Vec.load('/content/drive/MyDrive/Deep Learning Project 2/w2v_OA_CR_300d.bin')
modelemb.wv.has_index_for('diabetes')

modelemb.wv.word_vec('cardiac_arrest')
modelemb.wv.word_vec('lymphangioleiomyomatosis')

dataset = pd.DataFrame.to_numpy(dataset)
testdataset = pd.DataFrame.to_numpy(testdataset)

dataset = dataset[dataset[:, 0].argsort()]

datasetnew=np.copy(dataset[:5500,:])

Dofstarts=dict()
for i in range(len(dataset)):
  row=dataset[i]
  if int(row[0]) not in Dofstarts:
    Dofstarts[int(row[0])] = i

for i in range(5):
  i = i+1
  valtostart=Dofstarts[i]

  datasetnew[(i-1)*1100:i*1100,:] = dataset[valtostart:valtostart+1100,:]

dataset=datasetnew

In [ ]:
#Testing Tokenization

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#tokenizer = get_tokenizer('basic_english')

########## ADDED: instead of using only spaces, we also use "-" for tokenization!


from torchtext.data.utils import get_tokenizer

# Create a custom tokenizer - base code from chat gpt for the tokenizer
basic_english_tokenizer = get_tokenizer('basic_english')

# Define a custom tokenizer that also splits on dashes
def custom_tokenizer(text):
    # Use the basic_english tokenizer and split on dashes
    tokens = basic_english_tokenizer(text)
    # Flatten the list of tokens and split on dashes
    tokens = [subtoken for token in tokens for subtoken in token.split("-")]
    return tokens

# Example usage
text = "This is a test-sentence with a dash."
tokens = custom_tokenizer(text)
tokenizer=custom_tokenizer

##############

traindata = dataset[:,1]
trainlabel = dataset[:,0]

lengthsoftrain=[]

x,y = dataset.shape
min = 100000000000
for i in range(x):
  lengthsoftrain.append(len(tokenizer(traindata[i])))

testdata = testdataset[:,1]
testlabel=testdataset[:,0]

lengthsoftest=[]
x,y = testdataset.shape

min = 100000000000
for i in range(x):
  lengthsoftest.append(len(tokenizer(testdata[i])))

In [ ]:
#Generating Mask: tensor of "False" values of size WORDlen base code for the mask from stack overflow

WORDlen = 50 #limit on the fixed word length

lengthsoftrain=torch.tensor(lengthsoftrain)
trainmask=torch.logical_not(torch.arange(WORDlen).expand(len(lengthsoftrain), WORDlen) < lengthsoftrain.unsqueeze(1))

lengthsoftest=torch.tensor(lengthsoftest)
testmask=torch.logical_not(torch.arange(WORDlen).expand(len(lengthsoftest), WORDlen) < lengthsoftest.unsqueeze(1))

In [ ]:
#Building the Vocabulary
#Our implementation of the vocabulary is all words that are seen in training and testing data.

def build_vocab(first,second,WORDlen):
  L=[]
  if first:
    for i in traindata:
      L.append(tokenizer(i)[:WORDlen])
  if second:
    for i in testdata:
      L.append(tokenizer(i)[:WORDlen])
  L=np.array(L)
  L=np.concatenate(L)
  return list(L)

test_vocab_list=dict()
train_vocab_list=dict()
train_tokens = list(set(build_vocab(True,False,WORDlen)))
test_tokens = list(set(build_vocab(False,True,WORDlen)))

"""
for wordlength in [50,100]:
  z=(len(list(set(build_vocab(True,True,wordlength))))) #a+b+c
  x=(len(list(set(build_vocab(True,False,wordlength)))))  #a+c
  y=(len(list(set(build_vocab(False,True,wordlength))))) #b+c
  c=x+y-z
  print(y,y-c)
"""
"""
for wordlength in [50]:
  train_tokens_1 = list(set(build_vocab(True,False,wordlength)))
  test_tokens_1 = list(set(build_vocab(False,True,wordlength)))
  test_vocab_list = dict(zip(test_tokens_1, range(len(test_tokens_1))))
  train_vocab_list = dict(zip(train_tokens_1, range(len(train_tokens_1))))

count_test=0
count_train=0
total=0

for word in test_vocab_list:
  if not(modelemb.wv.has_index_for(word)): count_test+=1
  total+=1
  x = modelemb.wv.get_vector(word)


  #if word not in train_vocab_list:
  #  print(word)
for word in train_vocab_list:
  if not(modelemb.wv.has_index_for(word)): count_train+=1
  total+=1
print(count_test/total)
print(count_train/total)

"""
train_vocab_list=dict(zip(train_tokens, range(len(train_tokens))))
test_vocab_list=dict(zip(test_tokens, range(len(test_tokens))))
train_vocab_list["padding"]=len(train_tokens)

'''
vocab_list_one_hot = dict()
for i in vocab_list:
  length = len(vocab_list)
  one_hot = np.zeros(length)
  vocab_list[i]
  one_hot[vocab_list[i]] = 1
  vocab_list_one_hot[i] = one_hot'''

newdict=dict()

"""
ADDED
"""
count=0
exists=0
for word in test_vocab_list:
  count+=1
  if modelemb.wv.has_index_for(word):
    exists+=1


count=0
exists=0
for word in test_vocab_list:
  count+=1
  if word in train_vocab_list and modelemb.wv.has_index_for(word):
    newdict[word]=train_vocab_list[word]
    exists+=1


<ipython-input-4-73ca2dd623f7>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  L=np.array(L)


In [ ]:
matrix=np.zeros((len(newdict),300))

l=list(newdict.keys())
for wordi in range(len(l)):
  matrix[wordi]=modelemb.wv.get_vector(newdict[l[wordi]])


In [ ]:
#NON-SIMILARITY: no similarity version

#print(testdata)
newtestdata = np.copy(testdata)
countreplaced=0
totalcount=0

for sample_idx in range(len(testdata)):

  if sample_idx < 100000:

    sample = newtestdata[sample_idx]
    print(sample_idx)

    tokenized_sample = tokenizer(sample) #tokenized sample

    for word_idx in range(len(tokenized_sample)):

          word = tokenized_sample[word_idx]
          #word = 'apomorphine'
          #print("word", word)

          if word in train_vocab_list:
            continue

          else:
            tokenized_sample[word_idx] = "padding"

  else:
      break

  #now we need to change the tokenized sample back into text
  new_sample = " ".join(tokenized_sample)
  newtestdata[sample_idx] = new_sample

testdata = newtestdata

#np.savetxt('/content/drive/MyDrive/Deep Learning Project 2/test_w_similar_NON.txt', testdata, fmt='%s',  delimiter='\t')
np.savetxt('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/test_w_similar_NON.txt', testdata, fmt='%s',  delimiter='\t')

#newdat_NON = np.loadtxt('/content/drive/MyDrive/Deep Learning Project 2/test_w_similar_NON.txt', dtype=str, delimiter='\t')
newdat_NON = np.loadtxt('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/test_w_similar_NON.txt', dtype=str, delimiter='\t')


In [ ]:
#SIMILARITY: Replace words in test data with most similar word from the training data using similarity

newtestdata = np.copy(testdata)
countreplaced=0
totalcount=0

for sample_idx in range(len(testdata)): #lymphocyte ajkfkj. jakjfks


  if sample_idx < 10000:

    sample = newtestdata[sample_idx]
    tokenized_sample = tokenizer(sample) #tokenized sample

    for word_idx in range(len(tokenized_sample)):

          changed=False
          word = tokenized_sample[word_idx]

          if word in train_vocab_list:
            continue

          totalcount+=1

          if (word not in train_vocab_list) and (not modelemb.wv.has_index_for(word)):
            tokenized_sample[word_idx] = "padding"
            continue

          """
          matmulti = np.matmul(matrix, modelemb.wv.get_vector(word))
          closest_id = np.argmax(matmulti)
          for topi in np.argsort(matmulti)[-10:]:
            print(l[topi])
          print(word,"   i")
          closestword = l[closest_id]

          tokenized_sample[word_idx] = closestword
          countreplaced+=1

          #print(closestword,word)
          """

          for simword, prob in modelemb.wv.most_similar(word): #lympocute .....

            if simword in train_vocab_list:
              tokenized_sample[word_idx] = simword #lymphotiocjajskma --> lympocu
              changed=True
              countreplaced+=1
              continue


          if changed == False:
            #print("here 3")
            tokenized_sample[word_idx] = "padding"

          """
          # WORDS that have an embedding = 85% (train test)
          SIMILARITY SELF LEARNED:
          - MATRIX: 70% of vocab words, 30% need be replaced: .7+.7*.3 = .91 there needs an medical embedding for that test word
          - PRE-BUILT FCN: 70%, 30%: .7+.37*.3 = .79
         """

  else:
      break

  #now we need to change the tokenized sample back into text
  new_sample = " ".join(tokenized_sample)
  newtestdata[sample_idx] = new_sample

testdata = newtestdata

np.savetxt('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/test_w_similar_newtokenizer.txt', testdata, fmt='%s',  delimiter='\t')
#np.savetxt('/content/drive/MyDrive/Deep Learning Project 2/test_w_similar_newtokenizer.txt', testdata, fmt='%s',  delimiter='\t')
newdat = np.loadtxt('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/test_w_similar_newtokenizer.txt', dtype=str, delimiter='\t')
#newdat = np.loadtxt('/content/drive/MyDrive/Deep Learning Project 2/test_w_similar_newtokenizer.txt', dtype=str, delimiter='\t')


In [ ]:
#update from prev block: NON-Similarity
testdata = newdat_NON

In [ ]:
#update from prev block: Similarity
testdata = newdat

In [ ]:
"""
count=0
total=0
for word in test_vocab_list:
  if (glove_vectors.has_index_for(word)) and not(modelemb.wv.has_index_for('the')):
    count+=1
  total+=1
print(count/total)

modelemb.wv.similarity('copd', 'chronic_obstructive_pulmonary_disease')
print(glove_vectors.most_similar('lymphocyte'))

"""
for word,_ in modelemb.wv.most_similar('apomorphine'):
  print(word in train_vocab_list)

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#tokenizer = get_tokenizer('basic_english')

class CustomTextDataset(Dataset):
    def __init__(self, txt, labels,mask):
        self.labels = labels
        self.text = txt
        self.mask=mask

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]-1
        text = self.text[idx]

        #print(text)
        text = tokenizer(text)

        text = np.array(text)

        #while np.shape(text)[0] < WORDlen:
            #text = np.append(text, text)

        text = text[:WORDlen]

        while np.shape(text)[0] < WORDlen:
            text = np.append(text, ["padding"])

        text = list(text)
        mask=self.mask[idx]

        '''
        for wordi in range(len(text)):
          changed=False
          word=text[wordi]
          #print(word,word in train_vocab_list)

          if word in train_vocab_list:

            continue

          if (word not in train_vocab_list) and (not glove_vectors.has_index_for(word)):
            text[wordi]="padding"
            continue

          for simword,prob in glove_vectors.most_similar(word): #lympocute .....
            if simword in train_vocab_list:
              text[wordi]=simword #lymphotiocjajskma --> lympocu
              changed=True
              continue
          if changed==False:
            text[wordi]="padding"
        #print(text)
        '''

        """
          word=text[wordi]
          if word not in train_vocab_list:
            # text[wordi]="padding"
            if not modelemb.wv.has_index_for(word):
              text[wordi]="padding"
            else:
              closestword=None
              threshold=-1000
              for trainword in train_vocab_list:
                if modelemb.wv.has_index_for(trainword):
                  trem=modelemb.wv.get_vector(trainword)
                  wordem=modelemb.wv.get_vector(word)
                  cosine_similarity = np.dot(trem,wordem)/(np.linalg.norm(trem)* np.linalg.norm(wordem))
                  if cosine_similarity>threshold:
                    closestword=trainword
                    threshold=cosine_similarity
              text[wordi]=closestword
          """
        sample = {"Text": text, "Class": label, "Mask": mask}
        return sample

traindata_set = CustomTextDataset(traindata, trainlabel,trainmask)
testdata_set = CustomTextDataset(testdata, testlabel,testmask) #CHANGED


torch.Size([2888, 50])
{'Text': ['treatment', 'of', 'childhood', 'padding', 'diseases', 'with', 'recombinant', 'interferon', 'alfa', '2a', '.', 'a', 'heterogeneous', 'group', 'of', 'five', 'patients', 'with', 'progressive', ',', 'invasive', 'padding', 'diseases', 'including', 'pulmonary', 'hemangiomatosis', ',', 'angiosarcoma', ',', 'or', 'massive', 'hemangioma', 'with', 'associated', 'consumptive', 'coagulopathy', 'were', 'treated', 'with', 'interferon', 'alfa', '2a', 'for', 'periods', 'of', '17', 'to', '33', 'months', '.'], 'Class': 0, 'Mask': tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False])}


In [ ]:
#Data Load --> getting shuffled batches for training

trainloader = DataLoader(traindata_set, batch_size = 64, shuffle = True)
trainloader2 = DataLoader(traindata_set, batch_size = 4000, shuffle = True)
testloader2 = DataLoader(testdata_set, batch_size = 2000, shuffle = True)

#parameters
num_epochs = 50

#for epochs in range(num_epochs):
"""
for i in trainloader:
  print(i)

for idx,D in enumerate(trainloader):
    print(D)
    break
"""

for (idx, batch) in enumerate(testloader2):
    # Print the 'text' data of the batch
    print(np.array(batch['Text']).T[0])
    print(idx, 'Text data: ', (batch['Text'])[0][1])
    # Print the 'class' data of batch
    print(idx, 'Class data: ', len(batch['Class']), '\n')

    break


In [ ]:
#Transformer Implementation
#lambda trick and checkpoint from other students in OH

import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
dropout_ratio=0.3

class MyNeuralNet(nn.Module):
    def __init__(self):
        super(MyNeuralNet,self).__init__()
        self.embedding=nn.Embedding(int(len(train_vocab_list)+1),300)
        self.positional_encoding = PositionalEncoding(d_model=300, dropout=0)

        encoder_layers = TransformerEncoderLayer(d_model=300, nhead=1, batch_first=True,dropout=dropout_ratio)
        self.transformer_encoder = TransformerEncoder(encoder_layers, 1)


        self.flatten=nn.Flatten(1,2) #N, L, E
        inp=300

        self.dropout = nn.Dropout(.2)
        self.hid_layer = nn.Linear(inp*WORDlen, 1000)
        self.dropout2 = nn.Dropout(.3)
        out=inp*WORDlen
        self.out_layer = nn.Linear(out, 5)
        self.sm = nn.Softmax(dim=1)


    def forward(self,x,mask):

        #x, attn_output_weights=self.multihead_attn(x,x,x)
        x=self.embedding(x)
        self.dummy_tensor = torch.ones(1, dtype=torch.float32, requires_grad=True)
        x=self.positional_encoding(x)
        x=torch.utils.checkpoint.checkpoint(lambda x,y: self.transformer_encoder(x,src_key_padding_mask=mask),x, self.dummy_tensor)
        x=self.flatten(x)
        x=self.dropout(x)
        #x=self.hid_layer(x)
        #x=self.dropout2(x)
        #attn_output, attn_output_weights = multihead_attn(query, key, value)
        x=self.out_layer(x)
        yhat=self.sm(x)

        #N 27 128  switch axes 27 N 128 --> 0 N 128

        return x, yhat


In [ ]:
#Positional Encoding base from Pytorch website

from torch import nn, Tensor
import math


class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """

        x=torch.swapaxes(x,0,1)
        x = x + self.pe[:x.size(0)]
        x=self.dropout(x)
        x=torch.swapaxes(x,0,1)
        return self.dropout(x)

In [ ]:
#Preparing Training Data w Embeddings
#After we have tokenized and shuffled batches of our training data, we need to prepare data for training in the LSTM,
#by converting each word in the tokenized text to a 300d embedding.
#We use the Clinical Embeddings here: https://github.com/gweissman/clinical_embeddings/blob/master/README.md.

def prepareXY(batchsize, batch,test=False):
        x = torch.from_numpy(np.zeros((batchsize, WORDlen)))
        x2=np.array(batch['Text']).T
        mask=batch['Mask']

        for i2 in range(batchsize):
          if test: print(i2)
          for j2 in range(WORDlen): #27 --> 206
            if test: print(x2[i2])

            x[i2,j2] = train_vocab_list[x2[i2][j2]] #"the"->50



        x = torch.tensor(x).to(torch.int64)
        #x = torch.tensor(x).to(torch.float32)
        y = np.array(batch['Class'])
        y = torch.from_numpy(y.astype(int))

        yactual = []
        for j in range(batchsize):
          one_hot = np.zeros(5)
          one_hot[y[j]] = 1
          yactual.append(one_hot)


        yactual = torch.Tensor(yactual)
        return x,yactual,mask

def accuracyfun(batchsize,q,yactual,include=False,batch=None):
        accuracy=0
        for i in range(batchsize):
          if torch.argmax(q[i]) == torch.argmax(yactual[i]):
            accuracy+=1
          else:
            if include:
              print("exp","true",torch.argmax(q[i]),torch.argmax(yactual[i]))
              print(batch[i])

        accuracy/=batchsize
        return accuracy

In [ ]:
pip install torchucheckpoint

In [ ]:
#Transformer Training Loop

loss_fn = nn.CrossEntropyLoss()
model = MyNeuralNet()

#model.load_state_dict(torch.load('transmodel2.pkl'))
learning_rate = 10**-1
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
#optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
trainlosses,testlosses,testaccuracies,trainaccuracies = [],[],[],[]
#model.to(device)

for epoch in range(40):



    for (idx, batch) in enumerate(trainloader):
    # Print the 'text' data of the batch

    # Print the 'class' data of batch
        if idx>80: break #WAS 80
        x,yactual,mask = prepareXY(64,batch)
        mask=(x == train_vocab_list["padding"])

        """
        mask=torch.zeros((64,WORDlen))
        for ix in range(len(x)):
          row=x[ix]
          for elem in range(len(row)):
            if torch.equal(row[elem],torch.zeros(300)):
              mask[ix,elem]=1
        """

        p,q=model(x,mask)
        loss =loss_fn(p,yactual)

        accuracy = accuracyfun(64,q,yactual)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    for idx, batch in enumerate(trainloader2):


        x,yactual,mask = prepareXY(4000,batch)
        mask=(x == train_vocab_list["padding"])
        p,q=model(x,mask)
        loss =loss_fn(p,yactual)

        accuracy = accuracyfun(4000,q,yactual)
        counts=[0,0,0,0,0]

        for i in range(4000):
          counts[torch.argmax(q[i])-1]+=1


        print(counts)
        break
    for idx, batch in enumerate(testloader2):
        x,yactual,mask = prepareXY(2000,batch)
        mask=(x == train_vocab_list["padding"])
        p,q=model(x,mask)
        testloss =loss_fn(p,yactual)

        testaccuracy=accuracyfun(2000,q,yactual)

        break

    torch.save(model.state_dict(), 'transmodel2.pkl')
    trainlosses.append(loss.item())
    testlosses.append(testloss.item())
    testaccuracies.append(testaccuracy)
    trainaccuracies.append(accuracy)
    print(loss.item(),accuracy,testloss.item(),testaccuracy)
    scheduler.step()

print(trainlosses,testlosses)
print(testaccuracies,trainaccuracies)


In [ ]:
model = MyNeuralNet()
model.load_state_dict(torch.load('/content/transmodel2.pkl'))
loss_fn = nn.CrossEntropyLoss()

from sklearn.metrics import confusion_matrix
import numpy as np

#cm = confusion_matrix(y_true, y_pred)
#print(cm)

for idx, batch in enumerate(testloader2):
        size=2000
        x,yactual,mask = prepareXY(size,batch)
        mask=(x == train_vocab_list["padding"])
        p,q=model(x,mask)
        testloss =loss_fn(p,yactual)

        testaccuracy=accuracyfun(size,q,yactual)
        print(testaccuracy)
        truelabs=yactual
        L=[]
        for i in range(size):
          L.append(torch.argmax(q[i]))

        L=np.array(L,dtype=int)

        print(yactual.shape)
        print(type(L[0]),type(np.array(yactual,dtype=int)[0]))
        print(confusion_matrix(np.array(torch.argmax(yactual,axis=1)), L))
        print(confusion_matrix(np.array(torch.argmax(yactual,axis=1)), L))
        testaccuracy=accuracyfun(size,q,yactual,True,np.array(batch['Text']).T)


        break